# Food Delivery Orders Analysis


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
sns.set()


In [6]:
# Load dataset
df = pd.read_csv('orders.csv', parse_dates=['order_time'])
df['order_time'] = pd.to_datetime(df['order_time'])
df['hour'] = df['order_time'].dt.hour
df['date'] = df['order_time'].dt.date
df.head()


,order_id,customer_id,city,order_time,restaurant,cuisine,delivery_time_mins,order_value,rating,delivery_partner,hour,date
0,ORD100000,CUST1776,Ahmedabad,2025-04-09 06:02:04.894866,Restaurant_101,South Indian,32,243.67,5.0,Rider_60,6,2025-04-09
1,ORD100001,CUST1193,Delhi,2025-11-05 02:39:56.894866,Restaurant_196,Italian,10,760.91,5.0,Rider_440,2,2025-11-05
2,ORD100002,CUST2892,Hyderabad,2025-08-18 20:33:51.894866,Restaurant_40,Chinese,34,50.00,5.0,Rider_286,20,2025-08-18
3,ORD100003,CUST3169,Hyderabad,2025-06-30 18:12:42.894866,Restaurant_143,Continental,25,592.04,5.0,Rider_201,18,2025-06-30
4,ORD100004,CUST3411,Pune,2025-01-19 14:25:17.894866,Restaurant_28,Chinese,53,53.10,3.0,Rider_89,14,2025-01-19


## Basic checks & cleaning
- Check missing values
- Remove duplicates
- Fill missing ratings with median
- Remove extreme delivery_time outliers (> 180 mins)


In [9]:
df.info()
df = df.drop_duplicates()
df['rating'] = df['rating'].fillna(df['rating'].median())
df = df[df['delivery_time_mins'] <= 180]
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_id            12000 non-null  object        
 1   customer_id         12000 non-null  object        
 2   city                12000 non-null  object        
 3   order_time          12000 non-null  datetime64[ns]
 4   restaurant          12000 non-null  object        
 5   cuisine             12000 non-null  object        
 6   delivery_time_mins  12000 non-null  int64         
 7   order_value         12000 non-null  float64       
 8   rating              12000 non-null  float64       
 9   delivery_partner    12000 non-null  object        
 10  hour                12000 non-null  int32         
 11  date                12000 non-null  object        
dtypes: datetime64[ns](1), float64(2), int32(1), int64(1), object(7)
memory usage: 1.1+ MB


,order_time,delivery_time_mins,order_value,rating,hour
count,12000,12000.000000,12000.000000,12000.000000,12000.000000
mean,2025-05-23 23:44:09.361115904,42.838583,344.139114,4.474167,13.751083
min,2024-11-23 03:16:37.894866,8.000000,50.000000,1.000000,0.000000
25%,2025-02-21 21:27:01.644866048,31.000000,200.875000,4.000000,9.000000
50%,2025-05-23 06:27:01.894865920,41.000000,297.490000,5.000000,14.000000
75%,2025-08-22 18:24:50.394865920,53.000000,461.845000,5.000000,20.000000
max,2025-11-23 22:50:25.894866,174.000000,1273.830000,5.000000,23.000000
std,NaN,17.856024,188.096380,0.733060,6.546184


## KPIs
Calculate: Total Orders, Total Revenue, Average Order Value (AOV), Avg Delivery Time


In [10]:
total_orders = df['order_id'].nunique()
total_revenue = df['order_value'].sum()
aov = df.groupby('order_id')['order_value'].sum().mean()
avg_delivery = df['delivery_time_mins'].mean()
kpis = {'Total Orders': total_orders, 'Total Revenue': total_revenue, 'AOV': aov, 'Avg Delivery (mins)': avg_delivery}
kpis


{'Total Orders': 12000,
 'Total Revenue': np.float64(4129669.37),
 'AOV': np.float64(344.1391141666667),
 'Avg Delivery (mins)': np.float64(42.83858333333333)}

## EDA: Orders by city and peak hours


In [11]:
orders_by_city = df['city'].value_counts().reset_index()
orders_by_city.columns = ['city','orders']
orders_by_city


,city,orders
0,Delhi,2124
1,Bengaluru,2099
2,Mumbai,1657
3,Hyderabad,1442
4,Chennai,1438
5,Pune,1218
6,Kolkata,1033
7,Ahmedabad,989


In [12]:
orders_by_hour = df['hour'].value_counts().sort_index().reset_index()
orders_by_hour.columns = ['hour','orders']
orders_by_hour


,hour,orders
0,0,296
1,1,302
2,2,310
3,3,315
4,4,291
5,5,325
6,6,291
7,7,307
8,8,337
9,9,303


## EDA: Delivery times and ratings


In [13]:
avg_delivery_by_city = df.groupby('city')['delivery_time_mins'].mean().reset_index().sort_values('delivery_time_mins')
avg_rating_by_city = df.groupby('city')['rating'].mean().reset_index().sort_values('rating', ascending=False)
avg_delivery_by_city, avg_rating_by_city


(        city  delivery_time_mins
 0  Ahmedabad           35.591507
 7       Pune           36.918719
 5    Kolkata           39.608906
 4  Hyderabad           39.655340
 2    Chennai           41.487483
 1  Bengaluru           44.888042
 3      Delhi           47.326742
 6     Mumbai           49.122511,
         city    rating
 7       Pune  4.554187
 0  Ahmedabad  4.550051
 5    Kolkata  4.547919
 4  Hyderabad  4.502080
 2    Chennai  4.493741
 1  Bengaluru  4.451644
 3      Delhi  4.414313
 6     Mumbai  4.388051)

In [14]:
# Scatter: order value vs rating
fig = px.scatter(df.sample(1000), x='order_value', y='rating', title='Order Value vs Rating (sample)')
fig.show()


## Simple correlations and insights
- Check correlation between delivery time and rating


In [15]:
corr = df[['delivery_time_mins','order_value','rating']].corr()
corr


,delivery_time_mins,order_value,rating
delivery_time_mins,1.000000,0.014131,-0.221936
order_value,0.014131,1.000000,0.005733
rating,-0.221936,0.005733,1.000000


## Export cleaned data for dashboard
Save cleaned CSV as `data/orders_cleaned.csv` for Power BI or Excel import.


In [17]:
df.to_csv('orders_cleaned.csv', index=False)
print('Saved cleaned file: data/orders_cleaned.csv')


Saved cleaned file: data/orders_cleaned.csv
